# Using pre-trained Glove vectors with Scikit-learn and Keras

Pre-trained Glove word vectors can be downloaded here:

http://nlp.stanford.edu/data/glove.6B.zip

## Building the embeddings index

Loading the word vectors is pretty straight forward:

In [1]:
import os
import numpy as np

GLOVE_DIR = 'glove/'

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


Example usage of the embeddings_index:

In [2]:
embeddings_index['hockey']

array([ 0.1309    ,  0.011308  ,  0.85140002, -0.44016999, -0.92250001,
       -0.15672   ,  0.073091  , -0.10772   , -1.40629995, -0.2111    ,
       -0.35148999, -1.31570005,  0.38323   , -0.070724  , -0.3457    ,
        0.028638  ,  0.26027   ,  0.25404   , -0.38690999, -0.08535   ,
       -0.85211998,  1.89660001, -0.032885  ,  0.24192999,  0.28060001,
       -0.35846999,  0.13362999, -0.65781999,  0.89960998,  0.58815002,
       -0.59443998,  1.26090002, -0.57046002, -0.14374   ,  0.51617998,
       -1.11679995, -0.79299998,  1.04059994, -0.86282998, -0.81149   ,
        0.80037999,  0.19261   , -0.25137001, -1.15079999,  0.21164   ,
        0.75827998,  0.72103   , -0.72103   ,  0.48985001, -0.18956   ,
       -0.45517999, -0.37231001, -0.11526   ,  0.37957999, -0.047891  ,
       -1.74249995,  0.10357   ,  0.21019   ,  0.81901997,  1.21640003,
       -0.53884   ,  0.58485001, -0.26903999,  0.35712001,  0.30651999,
        0.22014999, -0.20929   ,  0.77614999, -0.21447   ,  0.36

## Loading the training and testing set

For training we will use the well known 20 newsgroups dataset.

In [3]:
import re
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian','comp.graphics','sci.med']

train_set = fetch_20newsgroups(subset='train',
                           categories=categories,
                           remove=('headers', 'footers', 'quotes'),
                           shuffle=True, 
                           random_state=42)

test_set = fetch_20newsgroups(subset='test',                          
                          categories=categories,
                          remove=('headers', 'footers', 'quotes'),
                          shuffle=True, 
                          random_state=42)

print("Train documents: %d"%(len(train_set.data)))
print("Test documents: %d"%(len(test_set.data)))

Train documents: 2257
Test documents: 1502


## Define the embedding vectorizer

We will define a Vectorizer class to be used with our pipeline. 

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(word2vec.values())))
        
    def fit(self, X, y):
        return self
        
    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] for w in texts.split() if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                        for texts in X])

## Support Vector Machine classifier

We will build a pipeline using our vectorizer and the SVM classifier (SVC) with a linear kernel.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

text_clf = Pipeline([('vectorizer', EmbeddingVectorizer(embeddings_index)),
                     ('clf', SVC(kernel="linear"))
])

text_clf.fit(train_set.data, train_set.target)
predicted = text_clf.predict(test_set.data)

print("Accuracy on the test set: %.2f%%" % (np.mean(predicted == test_set.target) * 100))

Accuracy on the test set: 75.63%


## Comparison with Bag of words approach

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

text_clf = Pipeline([('tfidf_vectorizer', TfidfVectorizer()),
                     ('clf', SVC(kernel="linear"))
])

text_clf.fit(train_set.data, train_set.target)
predicted = text_clf.predict(test_set.data)

print("Accuracy on the test set: %.2f%%" % (np.mean(predicted == test_set.target) * 100))

Accuracy on the test set: 80.23%


## Convolutional neural network in Keras

In [44]:
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import RMSprop

Prepare input embeddings and transform labels to binary class matrices:

In [148]:
vect = EmbeddingVectorizer(embeddings_index)

x_train = vect.transform(train_set.data)
x_test = vect.transform(test_set.data)

y_train = to_categorical(train_set.target) 
y_test = to_categorical(test_set.target)


x_train = np.reshape(x_train, (x_train.shape[0], 100, 1))
x_test = np.reshape(x_test, (x_test.shape[0], 100, 1))

x_train.shape

(2257, 100, 1)

Define our model:

In [213]:
from keras import optimizers

model = Sequential()
model.add(Conv1D(32, 3, activation='relu', input_shape=(100,1)))
model.add(MaxPooling1D(3))
model.add(Conv1D(64, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(1.0))
model.add(Dense(len(categories), activation='softmax', kernel_initializer='random_uniform'))

sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_164 (Conv1D)          (None, 98, 32)            128       
_________________________________________________________________
max_pooling1d_60 (MaxPooling (None, 32, 32)            0         
_________________________________________________________________
conv1d_165 (Conv1D)          (None, 30, 64)            6208      
_________________________________________________________________
global_average_pooling1d_58  (None, 64)                0         
_________________________________________________________________
dropout_58 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_58 (Dense)             (None, 4)                 260       
Total params: 6,596
Trainable params: 6,596
Non-trainable params: 0
_________________________________________________________________
None


Now let's train it an check the accuracy:

In [214]:
%%time

model.fit(x_train, y_train,
          batch_size=128,
          epochs=300,
          shuffle=True,
          validation_data=(x_test, y_test))

Train on 2257 samples, validate on 1502 samples
Epoch 1/300
2257/2257 [==============================] - 1s - loss: 1.3853 - acc: 0.2348 - val_loss: 1.3827 - val_acc: 0.2650
Epoch 2/300
2257/2257 [==============================] - 0s - loss: 1.3835 - acc: 0.2552 - val_loss: 1.3823 - val_acc: 0.2650
Epoch 3/300
2257/2257 [==============================] - 0s - loss: 1.3838 - acc: 0.2610 - val_loss: 1.3823 - val_acc: 0.2636
Epoch 4/300
2257/2257 [==============================] - 0s - loss: 1.3837 - acc: 0.2494 - val_loss: 1.3825 - val_acc: 0.2650
Epoch 5/300
2257/2257 [==============================] - 0s - loss: 1.3840 - acc: 0.2494 - val_loss: 1.3826 - val_acc: 0.2650
Epoch 6/300
2257/2257 [==============================] - 0s - loss: 1.3831 - acc: 0.2743 - val_loss: 1.3829 - val_acc: 0.2636
Epoch 7/300
2257/2257 [==============================] - 0s - loss: 1.3836 - acc: 0.2627 - val_loss: 1.3819 - val_acc: 0.2550
Epoch 8/300
2257/2257 [==============================] - 0s - loss: 1.

2257/2257 [==============================] - 0s - loss: 1.1853 - acc: 0.4426 - val_loss: 1.1689 - val_acc: 0.5253
Epoch 66/300
2257/2257 [==============================] - 0s - loss: 1.2788 - acc: 0.4081 - val_loss: 1.1781 - val_acc: 0.5220
Epoch 67/300
2257/2257 [==============================] - 0s - loss: 1.1543 - acc: 0.5055 - val_loss: 1.1837 - val_acc: 0.4334
Epoch 68/300
2257/2257 [==============================] - 0s - loss: 1.2548 - acc: 0.4307 - val_loss: 1.1800 - val_acc: 0.4521
Epoch 69/300
2257/2257 [==============================] - 0s - loss: 1.1509 - acc: 0.5281 - val_loss: 1.1349 - val_acc: 0.5493
Epoch 70/300
2257/2257 [==============================] - 0s - loss: 1.2786 - acc: 0.4400 - val_loss: 1.1867 - val_acc: 0.5100
Epoch 71/300
2257/2257 [==============================] - 0s - loss: 1.1490 - acc: 0.5179 - val_loss: 1.1756 - val_acc: 0.4874
Epoch 72/300
2257/2257 [==============================] - 0s - loss: 1.2266 - acc: 0.4572 - val_loss: 1.1811 - val_acc: 0.49

2257/2257 [==============================] - 0s - loss: 0.8383 - acc: 0.6597 - val_loss: 0.8989 - val_acc: 0.6285
Epoch 130/300
2257/2257 [==============================] - 0s - loss: 0.8740 - acc: 0.6283 - val_loss: 0.8889 - val_acc: 0.6451
Epoch 131/300
2257/2257 [==============================] - 0s - loss: 0.8218 - acc: 0.6526 - val_loss: 0.9090 - val_acc: 0.6158
Epoch 132/300
2257/2257 [==============================] - 0s - loss: 0.8771 - acc: 0.6318 - val_loss: 0.8973 - val_acc: 0.6378
Epoch 133/300
2257/2257 [==============================] - 0s - loss: 0.8305 - acc: 0.6540 - val_loss: 0.8990 - val_acc: 0.6138
Epoch 134/300
2257/2257 [==============================] - 0s - loss: 0.8364 - acc: 0.6447 - val_loss: 0.9016 - val_acc: 0.6352
Epoch 135/300
2257/2257 [==============================] - 0s - loss: 0.8051 - acc: 0.6664 - val_loss: 0.8198 - val_acc: 0.6605
Epoch 136/300
2257/2257 [==============================] - 0s - loss: 0.8120 - acc: 0.6708 - val_loss: 0.7988 - val_ac

2257/2257 [==============================] - 0s - loss: 0.6619 - acc: 0.7231 - val_loss: 0.8025 - val_acc: 0.6678
Epoch 194/300
2257/2257 [==============================] - 0s - loss: 0.7074 - acc: 0.6987 - val_loss: 0.8483 - val_acc: 0.6771
Epoch 195/300
2257/2257 [==============================] - 0s - loss: 0.7152 - acc: 0.7049 - val_loss: 0.7546 - val_acc: 0.6991
Epoch 196/300
2257/2257 [==============================] - 0s - loss: 0.6759 - acc: 0.7191 - val_loss: 0.7559 - val_acc: 0.6977
Epoch 197/300
2257/2257 [==============================] - 0s - loss: 0.6980 - acc: 0.7178 - val_loss: 0.7485 - val_acc: 0.6924
Epoch 198/300
2257/2257 [==============================] - 0s - loss: 0.6742 - acc: 0.7204 - val_loss: 0.7417 - val_acc: 0.6997
Epoch 199/300
2257/2257 [==============================] - 0s - loss: 0.6539 - acc: 0.7262 - val_loss: 0.9990 - val_acc: 0.6019
Epoch 200/300
2257/2257 [==============================] - 0s - loss: 0.6795 - acc: 0.7293 - val_loss: 0.7625 - val_ac

2257/2257 [==============================] - 0s - loss: 0.6173 - acc: 0.7421 - val_loss: 0.7444 - val_acc: 0.7024
Epoch 258/300
2257/2257 [==============================] - 0s - loss: 0.5943 - acc: 0.7576 - val_loss: 0.7315 - val_acc: 0.7204
Epoch 259/300
2257/2257 [==============================] - 0s - loss: 0.6131 - acc: 0.7457 - val_loss: 0.7242 - val_acc: 0.7124
Epoch 260/300
2257/2257 [==============================] - 0s - loss: 0.5754 - acc: 0.7599 - val_loss: 0.7463 - val_acc: 0.7037
Epoch 261/300
2257/2257 [==============================] - 0s - loss: 0.5781 - acc: 0.7572 - val_loss: 0.9133 - val_acc: 0.6631
Epoch 262/300
2257/2257 [==============================] - 0s - loss: 0.5879 - acc: 0.7603 - val_loss: 0.9249 - val_acc: 0.6391
Epoch 263/300
2257/2257 [==============================] - 0s - loss: 0.6030 - acc: 0.7528 - val_loss: 0.8601 - val_acc: 0.6498
Epoch 264/300
2257/2257 [==============================] - 0s - loss: 0.5941 - acc: 0.7603 - val_loss: 0.8229 - val_ac

In [215]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 72.17%


### Calculate real accuracy

Keras evaluate function sometimes outputs incorrect resutls so we should check again using predicted output.

In [216]:
predicted = model.predict(x_test)
print("Accuracy: %.2f%%" % (np.mean(np.equal(np.argmax(predicted, axis=-1), np.argmax(y_test, axis=-1))) * 100))

Accuracy: 72.17%
